# Assignment Python 1

In [ ]:
# Import Primary Modules:

import numpy as np # useful for many scientific computing in Python
import pandas as pd # primary data structure library
import matplotlib.pyplot as plt
from PIL import Image # converting images into arrays

The aim of this project is to introduce you to data visualization with Python as concrete and as consistent as possible.

Using what you’ve learned; download the London Crime Dataset from Kaggle.

 This dataset is a record of crime in major metropolitan areas, such as London, occurs in distinct patterns. This data covers the number of criminal reports by month, LSOA borough, and major/minor category from Jan 2008 - Des 2016.

This dataset contains:

- lsoa_code: this represents a policing area
- borough: the london borough for which the statistic is related
- major_category: the major crime category
- minor_category: the minor crime category
- value: the count of the crime for that particular borough, in that particular month
- year: the year of the summary statistic
- month: the month of the summary statistic

Formulate a question and derive a statistical hypothesis test to answer the question. You have to demostrate that you’re able to make decisions using data in a scientific manner. And the important things, Visualized the data. Examples of question can be:

    - What is the change in the number of crime incidents from 2011 to 2016?
    - What were the top 3 crimes incidents from 2011 to 2016?